In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K, Model
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback

from CRNN import KerasCrnn

tf.keras.backend.clear_session()

In [2]:
def ctc_custom_len(args):
    _true_labels, _pred_labels, _label_lens = args
    pred_len = [[_pred_labels.shape[2]]] * BATCH_SIZE
    pred_len = K.variable(pred_len)
    
    return K.ctc_batch_cost(_true_labels, K.softmax(_pred_labels), pred_len, _label_lens)

In [3]:
# Mel
IMG_HEIGHT = 480
IMG_WIDTH = 640
N_LABELS = 28
BATCH_SIZE = 64
LSTM_SIZE = 64
TRAINDATA_LIM = 51200
VALDATA_LIM = 4096
N_ITER = 25

# Define tensors for the waterfall inputs, label outputs, and predicted label outputs
true_labels = Input(name='feature_labels', shape=[N_LABELS], dtype='int32')
label_lens = Input(name='label_lens', shape=[1], dtype='int32')
imgs = Input(name='waterfall_input', shape=[IMG_HEIGHT, IMG_WIDTH, 1], dtype='float32')

# Construct CRNN architecture
mel_crnn = KerasCrnn(IMG_HEIGHT, N_LABELS, nh=LSTM_SIZE)
pred_labels = mel_crnn(imgs)

# Define loss
loss = Lambda(ctc_custom_len, output_shape=(0,), name='ctc')([true_labels, pred_labels, label_lens])

# Define entire model
fmodel = Model(inputs=[imgs, true_labels, label_lens], outputs=loss)
fmodel.compile(loss={'ctc': lambda true_labels, pred_labels: pred_labels}, optimizer=RMSprop( lr=0.01, epsilon=K.epsilon() ),  metrics=['accuracy'])

train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    directory='../data/mel/train/',
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=42
)
valid_datagen = ImageDataGenerator()
valid_generator = valid_datagen.flow_from_directory(
    directory='../data/mel/valid/',
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

class MelWeightsSaver(Callback):
    def on_epoch_end(self, epoch, logs={}):
        name = '../models/mel.h5'
        mel_crnn.save()
        print('Saved "%s"' % name)

tenbordlogs = TensorBoard(update_freq='batch')
model_saver = MelWeightsSaver()

fmodel.fit_generator(generator=train_datagen,
                    steps_per_epoch=int(TRAINDATA_LIM / BATCH_SIZE),
                    epochs=N_ITER,
                    validation_data=valid_datagen,
                    validation_steps=int(VALDATA_LIM / BATCH_SIZE),
                    initial_epoch=0, callbacks=[model_saver, tenbordlogs])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


AttributeError: 'ImageDataGenerator' object has no attribute 'shape'